In [1]:
import pandas as pd
import numpy as np

# 1035, 258
train = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-train-1035-from-spmm-contain-todense-over-3s-1293.csv')
test = pd.read_csv('/Users/bdlab/Desktop/sparse-matrix-multiplication/scenario-extraction/d-optimal/spmm-latency-traintest/train-test-csv/nonsquare-test-258-from-spmm-contain-todense-over-3s-1293.csv')

In [2]:
# Train + Valid
X_train = train[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_train = train['sp_smdm']

# Test
X_test = test[['lr','lc','rc','ld','rd','lnnz','rnnz','lr*lc','lc*rc','lr*rc','lr*lc*rc','ld*rd','lr*rc*ld*rd','lr*lc*rc*ld*rd','lnnz*rnnz']] 
y_test = test['sp_smdm']

In [3]:
from sklearn.metrics import r2_score, mean_squared_error

# Metric
def mean_absolute_percentage_error(y_test, y_pred):
    y_test, y_pred = np.array(y_test), np.array(y_pred)
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100

def custom_scoring(real, pred):
    rmse = np.sqrt(mean_squared_error(real, pred))
    r2 = r2_score(real, pred)
    mape = mean_absolute_percentage_error(real, pred)
    return mape

In [4]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

# 탐색 대상 함수 (XGBRegressor)
def XGB(
max_depth,
learning_rate, 
n_estimators, 
# min_child_weight, 
# subsample,
# _lambda,
# gamma ,
# colsample_bytree, 
# _alpha,
# silent=True, 
n_jobs=-1):
    
    # 모델 정의
    model = xgb.XGBRegressor( 
objective = 'reg:squarederror',
max_depth=int(max_depth),
learning_rate=learning_rate,
n_estimators=int(n_estimators)
# min_child_weight=min_child_weight,
# subsample=subsample,
# reg_lambda=_lambda,        
# gamma=gamma,
# colsample_bytree=colsample_bytree, 
# reg_alpha=_alpha,
# n_jobs=n_jobs        
                              )
    
    # bayesian optimization을 통해 파라미터를 받아
    # Train을 Train + Validation으로 나눠 cross-validation 성능 확인
    kfold = KFold(n_splits=5, shuffle = True, random_state=0)

    # cross-validation 평균 성능 성능 확인
    score = cross_val_score(model,
                            X_train, 
                            y_train, 
                            cv=kfold,
                            scoring=make_scorer(custom_scoring,greater_is_better=False),
                            n_jobs=-1
                           ).mean()
    
    return score

In [5]:
from bayes_opt import BayesianOptimization

# # 하이퍼파라미터 정의
# pbounds = {
# 'max_depth': (12, 16), 
# 'learning_rate': (0.01, 0.3),
# 'n_estimators': (50, 150),
# 'min_child_weight': (0.1, 1),
# 'subsample': (0.5, 1), 
# '_lambda' : (0.1,1),    
# #'gamma': (0, 0.3),     
# #'colsample_bytree' :(0.5, 1)
# #'_alpha' : (0,1)           
#                       }

# 하이퍼파라미터 정의
pbounds = {
'max_depth': (5, 15), 
'learning_rate': (0.01, 0.3),
'n_estimators': (100, 6000),
#'min_child_weight': (0.1, 1.5),
#'subsample': (0.5, 1), 
#'_lambda' : (0.1,1),    
#'gamma': (0.1, 1),     
#'colsample_bytree' :(0.4, 1),
#'_alpha' : (0,0.1)
                      }


# Bayesian optimization 객체 생성
bo=BayesianOptimization(f=XGB, pbounds=pbounds, verbose=2, random_state=1 )    

# 메소드를 이용해 최대화 과정 수행 (파라미터 넣고 목적함수 값 출력하고)
bo.maximize(init_points=2, n_iter=50, acq='ei', xi=0.01)

# 뽑힌 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
|  1        | -22.32    |  0.1309   |  12.2     |  100.7    |
|  2        | -21.85    |  0.09768  |  6.468    |  644.8    |
|  3        | -21.42    |  0.09932  |  7.383    |  645.8    |
|  4        | -23.01    |  0.244    |  9.095    |  648.4    |
|  5        | -21.07    |  0.02262  |  5.27     |  647.7    |
|  6        | -21.86    |  0.1703   |  10.01    |  643.3    |
|  7        | -21.3     |  0.06473  |  5.035    |  650.3    |
|  8        | -21.71    |  0.1337   |  5.923    |  656.0    |
|  9        | -21.2     |  0.03837  |  6.28     |  660.1    |
|  10       | -23.22    |  0.2907   |  7.669    |  662.2    |
|  11       | -21.6     |  0.09433  |  6.656    |  650.2    |
|  12       | -20.9     |  0.03752  |  5.946    |  658.7    |
|  13       | -22.34    |  0.1484   |  10.08    |  656.9    |
|  14       | -20.8     |  0.05082  |  6.394    |  1.969e+0 |
|  15   

KeyboardInterrupt: 

In [ ]:
import xgboost as xgb

# Train + Valid cross-validation을 거친, 최적의 하이퍼파라미터를 사용
best_model = xgb.XGBRegressor(
objective = 'reg:squarederror',
max_depth=int(bo.max['params']['max_depth']),
learning_rate=bo.max['params']['learning_rate'],
n_estimators=int(bo.max['params']['n_estimators']),
# min_child_weight=bo.max['params']['min_child_weight'],
# subsample=bo.max['params']['subsample'],
# reg_lambda = bo.max['params']['_lambda'],    
# gamma=bo.max['params']['gamma'],
# colsample_bytree=bo.max['params']['colsample_bytree'],
# reg_alpha = bo.max['params']['_alpha'],
# n_jobs=-1
                             )
# 모델 훈련
best_model.fit(X_train, y_train)

In [ ]:
# 최적의 하이퍼파라미터 값 확인
print("{}\n".format(bo.max))

# 훈련데이터 예측
y_train_pred = best_model.predict(X_train)
print("-------- 훈련데이터 예측 --------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_train, y_train_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_train, y_train_pred)))
print("\n")

# 검증데이터 예측
print("-------- 검증데이터 예측 --------------------------")
print("mape : {}%".format(-bo.max['target']))
print("\n")

# 테스트데이터 예측
y_pred = best_model.predict(X_test)
print("-------- 테스트데이터 예측 -------------------------")
print("rmse : {}".format(np.sqrt(mean_squared_error(y_test, y_pred))))
print("mape : {}%".format(mean_absolute_percentage_error(y_test, y_pred)))
print("\n")

In [ ]:
# mape_list = {}
# # 예측값, 실제값을 확인하며 mape 계산 후 mape_list에 삽입 
# for idx,value in enumerate(y_test):
#     mape_temp = {}
#     predicate = int(y_pred[idx])
#     mape = abs((value - predicate) / value) * 100
#     mape_temp['pred'] = predicate
#     mape_temp['real'] = value
#     mape_temp['mape'] = mape
#     mape_list[idx] = mape_temp
# mape_list_sort = sorted(mape_list.values(), key=lambda x:(x['mape']), reverse=True)
# mape_list_sort  

In [ ]:
xgb.plot_importance(best_model)